In [5]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)


In [6]:
accepted_raw=pd.read_csv('accepted_2007_to_2018Q4.csv.gz')

C:\Users\Laptop2\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
accepted_raw.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.00,0.00,4421.723917,4421.72,3600.00,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.00,0.00,25679.660000,25679.66,24700.00,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN,N

In [10]:
sample=accepted_raw.sample(3000)

In [23]:
import numpy as np
from sklearn.impute import SimpleImputer

def data_processing(data, one_hot_encoder=False,Label_encoder=False):

    data=data.copy()
    
    #Drop rows with missing information in issue_d.
#     row_to_drop=data[data.issue_d.isna()].index.tolist()
#     data=data.drop(data.index[row_to_drop])
    
    #drop irrevalent
    drop_list=['id','member_id',"funded_amnt","funded_amnt_inv",'desc','title','issue_d',
              'int_rate','installment','loan_status','pymnt_plan',
              'url','last_pymnt_d','last_pymnt_amnt','next_pymnt_d','policy_code']

    data=data.drop(drop_list,axis=1)
      
    data.emp_title = data.emp_title.fillna('None')
    
    #I think term should be categorical, not numerical
    #data['term'] = data.term.apply(lambda x: x.split()[0])

    data['emp_length'] = data['emp_length'].str.extract('(\d+)')
    data.emp_length = data.emp_length.fillna(0)
    
    ''' when annual_inc is missing, 
    employment title is missing too. 
    However, these loans are marked s grade A. 
    fill 0 for now
    '''

    data.emp_title = data.emp_title.fillna('None')
    data.zip_code =data.zip_code.fillna('None')

           
    # Drop rows with an insignificant amount of number of missing values
    #acc_now_delinq should not be dropped
    drop_rows = list(data[['loan_amnt','annual_inc','earliest_cr_line','delinq_2yrs',
                      'earliest_cr_line','inq_last_6mths']])

    for drop_row in drop_rows:
        data = data.dropna(axis=0, subset=[drop_row])
    
    data.dti = data.dti.fillna(data.revol_bal / data.annual_inc)
        
    # Time between earliest credit line and last credit pull

    data['earliest_cr_line'] = pd.to_datetime(data['earliest_cr_line'])
    data['last_credit_pull_d']=pd.to_datetime(data['earliest_cr_line'])
    data['cr_line_months'] = ((data['last_credit_pull_d'] - data['earliest_cr_line'])/np.timedelta64(1, 'M'))
    data['cr_line_months'] = data['cr_line_months'].astype(int)
    data=data.drop(['earliest_cr_line','last_credit_pull_d'], axis=1)
        
    #Missing value
    data.inq_last_6mths=data.inq_last_6mths.fillna(0)
    data.mths_since_last_record = data.mths_since_last_record.fillna(0)
    data.mths_since_last_delinq = data.mths_since_last_delinq.fillna(0)
    data.open_acc=data.open_acc.fillna(0)
    data.revol_util=data.revol_util.fillna(0)
    
    #collections_12_mths_ex_med
    data.collections_12_mths_ex_med=data.collections_12_mths_ex_med.fillna(0)
    data.mths_since_last_major_derog=data.mths_since_last_major_derog.fillna(0)
    
    
    #annual income should not be dropped, fill with -999
    data.annual_inc_joint=data.annual_inc_joint.fillna(-999)
    
    #dti_join should not be dropped
    data.dti_joint=data.dti_joint.fillna(-999)
    
    #Missing value
    data.verification_status_joint=data.verification_status_joint.fillna('None')
    
    #Missing value
    data.tot_coll_amt=data.tot_coll_amt.fillna(0)
    data.tot_cur_bal=data.tot_cur_bal.fillna(0)
    data.open_acc_6m=data.open_acc_6m.fillna(0)
    data.open_act_il=data.open_act_il.fillna(0)
    data.open_il_12m=data.open_il_12m.fillna(0)
    data.open_il_24m=data.open_il_24m.fillna(0)
    data.mths_since_rcnt_il =data.mths_since_rcnt_il.fillna(0)
    data.total_bal_il=data.total_bal_il.fillna(0)
    data.il_util=data.il_util.fillna(0)
    data.open_rv_12m=data.open_rv_12m.fillna(0)
    data.open_rv_24m=data.open_rv_24m.fillna(0)
    data.max_bal_bc=data.max_bal_bc.fillna(-999)
    data.all_util=data.all_util.fillna(-999)
    data.total_rev_hi_lim=data.total_rev_hi_lim.fillna(-999)
    

# inq_fi  
# : [ 3.  0.  2.  1.  5.  4.  6.  8. 16.  7.  9. 10. 13. 11. 17. 12. 14. nan
#  15. 18. 21. 19. 22. 38. 20. 32. 23. 27. 28. 24. 29. 31. 25. 48.] 866162
    data.inq_fi=data.inq_fi.fillna(-999)
    
# total_cu_tl  
# : [  1.   0.   5.   2.   3.   4.   6.   8.   9.   7.  13.  12.  10.  11.
#   14.  18.  16.  17.  15.  19.  24.  21.  22.  28.  23.  20.  nan  29.
#   30.  33.  27.  25.  35.  32.  26.  31.  52.  39.  34.  40.  38.  36.
#   46.  37. 111.  54.  44.  49.  45.  42.  68.  51.  71.  43.  77.  60.
#   47.  79.  41.  48.  56.  55.  65.] 866163
    data.total_cu_tl=data.total_cu_tl.fillna(-999)
    
# inq_last_12m  
# : [ 4.  6.  1.  0.  3.  2.  5.  9.  7.  8. 10. 12. 14. 16. 13. 20. 11. 15.
#  19. 18. 26. 21. 17. 30. 25. nan 22. 32. 24. 27. 23. 33. 31. 39. 45. 46.
#  29. 40. 35. 41. 49. 38. 36. 28. 34. 51. 67. 42. 37.] 866163
    data.inq_last_12m=data.fillna(-999)
    
# acc_open_past_24mths  
# : [ 4.  6.  2. 10.  0.  7.  5.  9.  3.  1. 17. 13. 12.  8. 11. 14. 19. 24.
#  18. 15. 16. 25. 31. 32. 20. 22. 21. 23. 26. 50. 40. 29. 64. 27. 35. 34.
#  47. 28. 39. 33. 36. 41. 38. 48. 30. 42. 37. nan 46. 54. 49. 56. 45. 43.
#  53. 44. 55. 61.] 50063
    data.acc_open_past_24mths=data.acc_open_past_24mths.fillna(0)
    
# avg_cur_bal  
# : [20701.  9733. 31617. ... 79516. 85158. 72812.] 70379
    data.avg_cur_bal=data.avg_cur_bal.fillna(0)
    
# bc_open_to_buy  
# : [ 1506. 57830.  2737. ... 79096. 50160. 73489.] 74968
    data.bc_open_to_buy=data.bc_open_to_buy.fillna(0)


# bc_util  
# : [ 37.2  27.1  55.9 ... 162.  194.9 193. ] 76104
    data.bc_util=data.bc_util.fillna(0)
    
    #drop irrevalent columns
    columns_to_drop=['chargeoff_within_12_mths','delinq_amnt',]
    
    data=data.drop(columns_to_drop, axis=1)

    data.mo_sin_old_il_acct=data.mo_sin_old_il_acct.fillna(0)
    data.mo_sin_old_rev_tl_op =data.mo_sin_old_rev_tl_op .fillna(0)
    data.mo_sin_rcnt_rev_tl_op=data.mo_sin_rcnt_rev_tl_op.fillna(0)
    data.mo_sin_rcnt_tl=data.mo_sin_rcnt_tl.fillna(0)
    data.mort_acc=data.mort_acc.fillna(0)
    data.mths_since_recent_bc=data.mths_since_recent_bc.fillna(0)
    data.mths_since_recent_bc_dlq=data.mths_since_recent_bc_dlq.fillna(0)
    data.mths_since_recent_inq=data.mths_since_recent_inq.fillna(0)
    data.mths_since_recent_revol_delinq=data.mths_since_recent_revol_delinq.fillna(0)
    data.num_accts_ever_120_pd=data.num_accts_ever_120_pd.fillna(0)
    data.num_actv_bc_tl=data.num_actv_bc_tl.fillna(0)
    data.num_actv_rev_tl=data.num_actv_rev_tl.fillna(0)
    data.num_bc_sats=data.num_bc_sats.fillna(0)
    data.num_bc_tl=data.num_bc_tl.fillna(0)
    data.num_il_tl=data.num_il_tl.fillna(0)
    data.num_op_rev_tl=data.num_op_rev_tl.fillna(0)
    data.num_rev_accts=data.num_rev_accts.fillna(0)
    data.num_rev_tl_bal_gt_0=data.num_rev_tl_bal_gt_0.fillna(0)
    data.num_sats=data.num_sats.fillna(0)
    data.num_tl_120dpd_2m=data.num_tl_120dpd_2m.fillna(0)
    data.num_tl_30dpd=data.num_tl_30dpd.fillna(0)
    data.num_tl_90g_dpd_24m=data.num_tl_90g_dpd_24m.fillna(0)
    data.num_tl_op_past_12m=data.num_tl_op_past_12m.fillna(0)
    data.pct_tl_nvr_dlq=data.pct_tl_nvr_dlq.fillna(0)
    data.percent_bc_gt_75=data.percent_bc_gt_75.fillna(0)
    data.pub_rec_bankruptcies=data.pub_rec_bankruptcies.fillna(0)
    data.tax_liens=data.tax_liens.fillna(0)
    data.tot_hi_cred_lim=data.tot_hi_cred_lim.fillna(0)
    data.total_bal_ex_mort=data.total_bal_ex_mort.fillna(0)
    data.total_bc_limit=data.total_bc_limit.fillna(0)
    data.total_il_high_credit_limit=data.total_il_high_credit_limit.fillna(0)
    data.revol_bal_joint=data.revol_bal_joint.fillna(0)

    
    
    column_secondary_applicant=['sec_app_fico_range_low','sec_app_fico_range_high',
                               "sec_app_earliest_cr_line",'sec_app_inq_last_6mths',
                               'sec_app_mort_acc','sec_app_open_acc','sec_app_revol_util',
                               'sec_app_open_act_il',"sec_app_num_rev_accts",'sec_app_chargeoff_within_12_mths',
                               'sec_app_collections_12_mths_ex_med',"sec_app_mths_since_last_major_derog",
                               ]

    for column in column_secondary_applicant:
        data[column]=data[column].fillna(-999)
    

    #Drop Columns after loan issued
    columns_after_loan_issued=['hardship_flag',
     'hardship_type',
     'hardship_reason',
     'hardship_status',
     'deferral_term',
     'hardship_amount',
     'hardship_start_date',
     'hardship_end_date',
     'payment_plan_start_date',
     'hardship_length',
     'hardship_dpd',
     'hardship_loan_status',
     'orig_projected_additional_accrued_interest',
     'hardship_payoff_balance_amount',
     'hardship_last_payment_amount',
     'disbursement_method',
     'debt_settlement_flag',
     'debt_settlement_flag_date',
     'settlement_status',
     'settlement_date',
     'settlement_amount',
     'settlement_percentage',
     'settlement_term']

    data=data.drop(columns_after_loan_issued, axis=1)

    
    #Imputig ordinal data
    data=data.replace({'term':{' 36 months':36, ' 60 months': 60},
                 'grade':{'G':0,'F':1, 'E':2, 'D':3,'C':4,'B':5,'A':6},
                 'sub_grade': {'G5':0, 'G4':1,'G3':2,'G2':3,'G1':4,
                              'F5':5, 'F4':6,'F3':7,'F2':8,'F1':9,
                              'E5':10, 'E4':11,'E3':12,'E2':13,'E1':14,
                              'D5':15, 'D4':16,'D3':17,'D2':18,'D1':19,
                              'C5':20, 'C4':21,'C3':22,'C2':23,'C1':24,
                              'B5':25, 'B4':26,'B3':27,'B2':28,'B1':29,
                              'A5':30, 'A4':31,'A3':32,'A2':33,'A1':34}   })    
        
    #Change categorical data to numerical
    data.emp_length= data.emp_length.apply(lambda x: int(x))  
       
#date???--need to reconsider
    data.sec_app_earliest_cr_line=pd.to_datetime(data.sec_app_earliest_cr_line)
     
        
    if one_hot_encoder:
        print (data.select_dtypes(include=['object']).columns.to_list())
        data = pd.get_dummies(data, columns=data.select_dtypes(include=['object']).columns, drop_first=True)
    
    #get label encoder. categorical data change to numerical values
    if Label_encoder:
        le = LabelEncoder()
        categorical_ordinal_col=data.select_dtypes(include=['object']).columns.to_list()
        data[categorical_ordinal_col]=data[categorical_ordinal_col].apply(lambda col: le.fit_transform(col))
    
    
        
    return data

In [2]:
processed=data_processing(sample,one_hot_encoder=True, Label_encoder=False)

NameError: name 'data_processing' is not defined